# 06 - Pipelines and ColumnTransformer

Pipelines bundle preprocessing and modeling into a single object, preventing data leakage and simplifying deployment. This notebook shows how to build production-ready ML workflows with scikit-learn.

## Learning Objectives

By the end of this notebook, you will be able to:

- Explain why pipelines prevent data leakage and simplify workflows
- Build sklearn `Pipeline` objects that chain preprocessors and models
- Use `ColumnTransformer` to apply different transforms to numeric vs categorical columns
- Combine `ColumnTransformer` + `LogisticRegression` into a single pipeline
- Use `make_column_selector` for automatic column selection
- Integrate pipelines with `cross_val_score`
- Compare manual preprocessing vs pipeline approaches

## Prerequisites

- Train/test splitting (Notebooks 01-03)
- Feature engineering basics (Notebook 04)
- Scaling, encoding, and imputation (Notebook 05)
- Basic understanding of Logistic Regression (used as the model in our pipeline)

## Table of Contents

1. [Why Pipelines?](#1-why-pipelines)
2. [sklearn Pipeline Basics](#2-sklearn-pipeline-basics)
3. [ColumnTransformer: Different Transforms for Different Columns](#3-columntransformer-different-transforms-for-different-columns)
4. [Full Pipeline: ColumnTransformer + LogisticRegression](#4-full-pipeline-columntransformer--logisticregression)
5. [Using make_column_selector](#5-using-make_column_selector)
6. [Pipelines with cross_val_score](#6-pipelines-with-cross_val_score)
7. [Manual Preprocessing vs Pipeline: A Comparison](#7-manual-preprocessing-vs-pipeline-a-comparison)
8. [Common Mistakes](#8-common-mistakes)
9. [Exercise](#9-exercise)

---

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

print("Setup complete.")

---

## 1. Why Pipelines?

**Without pipelines**, a typical preprocessing workflow looks like this:

```python
# Manual approach - error-prone
imputer = SimpleImputer()
X_train_imp = imputer.fit_transform(X_train)
X_test_imp = imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imp)
X_test_scaled = scaler.transform(X_test_imp)

model = LogisticRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
```

**Problems with the manual approach:**
- Easy to accidentally call `fit_transform` on test data
- Multiple objects to manage and serialize for production
- Cannot be used directly with `cross_val_score` (leakage within folds)
- Code becomes a tangled mess with many preprocessing steps

**With pipelines:**
- Single object: `pipeline.fit(X_train, y_train)` and `pipeline.predict(X_test)`
- Leakage is impossible - each fold refits automatically
- Easy to serialize (`joblib.dump(pipeline, 'model.pkl')`)
- Clean, readable code

---

## 2. sklearn Pipeline Basics

A `Pipeline` chains multiple steps sequentially. Each step (except the last) must be a **transformer** (has `fit` and `transform`). The last step can be a transformer or an **estimator** (has `fit` and `predict`).

In [ ]:
# Create simple numeric dataset
np.random.seed(42)
n = 200
X_num = pd.DataFrame({
    'feature1': np.where(np.random.random(n) < 0.1, np.nan, np.random.normal(50, 15, n)),
    'feature2': np.where(np.random.random(n) < 0.1, np.nan, np.random.normal(100, 30, n)),
})
y_num = (X_num['feature1'].fillna(50) + X_num['feature2'].fillna(100) > 155).astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X_num, y_num, test_size=0.2, random_state=42
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Missing values in train:\n{X_train.isnull().sum()}")

In [ ]:
# Build a Pipeline: impute -> scale -> model
pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(random_state=42))
])

# One call does it all: impute, scale, fit the model
pipe.fit(X_train, y_train)

# One call: impute, scale, predict (using train-fitted parameters)
y_pred = pipe.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Pipeline accuracy: {accuracy:.4f}")
print(f"\nPipeline steps: {[step[0] for step in pipe.steps]}")

In [ ]:
# Access individual steps
print("Imputer statistics (medians learned from train):")
print(f"  {pipe.named_steps['imputer'].statistics_}")
print()
print("Scaler parameters (learned from imputed train):")
print(f"  Mean: {pipe.named_steps['scaler'].mean_.round(2)}")
print(f"  Std:  {pipe.named_steps['scaler'].scale_.round(2)}")
print()
print("Model coefficients:")
print(f"  {pipe.named_steps['model'].coef_.round(4)}")

In [ ]:
# Shortcut: make_pipeline (auto-generates step names)
pipe_short = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler(),
    LogisticRegression(random_state=42)
)

pipe_short.fit(X_train, y_train)
print(f"make_pipeline accuracy: {pipe_short.score(X_test, y_test):.4f}")
print(f"Auto-named steps: {[step[0] for step in pipe_short.steps]}")

---

## 3. ColumnTransformer: Different Transforms for Different Columns

Real datasets have **mixed types**: numeric columns need scaling, categorical columns need encoding. `ColumnTransformer` applies different transformers to different column subsets in parallel.

```
ColumnTransformer
  |-- numeric_cols  -> Imputer -> Scaler
  |-- categorical_cols -> Imputer -> OneHotEncoder
  |-- (remainder: drop or passthrough)
```

In [ ]:
# Create a mixed dataset
np.random.seed(42)
n = 300

df_mixed = pd.DataFrame({
    'age': np.where(np.random.random(n) < 0.1, np.nan, np.random.normal(40, 12, n)),
    'income': np.where(np.random.random(n) < 0.08, np.nan, np.random.normal(55000, 15000, n)),
    'score': np.random.normal(70, 10, n),
    'department': np.random.choice(['engineering', 'marketing', 'sales', 'hr'], n),
    'education': np.random.choice(['high_school', 'bachelors', 'masters', 'phd'], n),
})

# Target: high performer
y_mixed = ((df_mixed['score'].fillna(70) > 72) & 
           (df_mixed['age'].fillna(40) > 35)).astype(int)

print("Mixed dataset:")
print(df_mixed.head())
print(f"\nDtypes:\n{df_mixed.dtypes}")
print(f"\nMissing values:\n{df_mixed.isnull().sum()}")

In [ ]:
# Define column groups
numeric_features = ['age', 'income', 'score']
categorical_features = ['department', 'education']

# Define sub-pipelines for each column type
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine with ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

print("ColumnTransformer created with:")
print(f"  Numeric pipeline:     imputer(median) -> StandardScaler")
print(f"  Categorical pipeline: imputer(mode) -> OneHotEncoder")

In [ ]:
# Split and apply
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    df_mixed, y_mixed, test_size=0.2, random_state=42
)

X_train_processed = preprocessor.fit_transform(X_train_m)
X_test_processed = preprocessor.transform(X_test_m)

print(f"Train shape after preprocessing: {X_train_processed.shape}")
print(f"Test shape after preprocessing:  {X_test_processed.shape}")
print()

# Get feature names
feature_names = preprocessor.get_feature_names_out()
print(f"Feature names ({len(feature_names)} total):")
for name in feature_names:
    print(f"  {name}")

---

## 4. Full Pipeline: ColumnTransformer + LogisticRegression

The real power comes from chaining the `ColumnTransformer` (preprocessor) with a model into a single `Pipeline`.

In [ ]:
# Full end-to-end pipeline
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

# One call: impute + scale + encode + fit model
full_pipeline.fit(X_train_m, y_train_m)

# One call: impute + scale + encode + predict
y_pred_m = full_pipeline.predict(X_test_m)

accuracy_m = accuracy_score(y_test_m, y_pred_m)
print(f"Full pipeline accuracy: {accuracy_m:.4f}")
print(f"\nPipeline structure:")
print(full_pipeline)

In [ ]:
# Inspect the learned model coefficients
model_coefs = full_pipeline.named_steps['classifier'].coef_[0]
feature_names = full_pipeline.named_steps['preprocessor'].get_feature_names_out()

coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': model_coefs
}).sort_values('Coefficient', ascending=False)

print("Feature importance (Logistic Regression coefficients):")
print(coef_df.to_string(index=False))

In [ ]:
# Visualize feature importance
fig, ax = plt.subplots(figsize=(10, 5))
colors = ['steelblue' if c > 0 else 'salmon' for c in coef_df['Coefficient']]
ax.barh(coef_df['Feature'], coef_df['Coefficient'], color=colors, edgecolor='black')
ax.set_xlabel('Coefficient Value')
ax.set_title('Logistic Regression Coefficients (Full Pipeline)', fontsize=13)
ax.axvline(0, color='black', linewidth=0.8)
plt.tight_layout()
plt.show()

---

## 5. Using make_column_selector

Instead of manually listing column names, `make_column_selector` selects columns by dtype automatically. This makes pipelines more flexible and reusable.

In [ ]:
# Automatic column selection based on dtype
preprocessor_auto = ColumnTransformer([
    ('num', numeric_transformer, make_column_selector(dtype_include=np.number)),
    ('cat', categorical_transformer, make_column_selector(dtype_include=object))
])

auto_pipeline = Pipeline([
    ('preprocessor', preprocessor_auto),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

auto_pipeline.fit(X_train_m, y_train_m)
y_pred_auto = auto_pipeline.predict(X_test_m)

print(f"Auto-selector pipeline accuracy: {accuracy_score(y_test_m, y_pred_auto):.4f}")
print()
print("Numeric columns selected:", 
      list(X_train_m.select_dtypes(include=np.number).columns))
print("Categorical columns selected:", 
      list(X_train_m.select_dtypes(include=object).columns))

In [ ]:
# Using manual column lists vs make_column_selector
print("Manual column lists:")
print("  + Explicit, clear what columns are used")
print("  - Must update if columns change")
print()
print("make_column_selector:")
print("  + Automatically adapts to new columns of the same type")
print("  - Less explicit, may pick up unwanted columns")
print()
print("Recommendation: Use manual lists in production for safety.")
print("Use selectors for quick prototyping and exploration.")

---

## 6. Pipelines with cross_val_score

This is where pipelines truly shine. When used with `cross_val_score`, **each fold** refits the entire pipeline from scratch. This means:
- Imputer statistics are computed per fold (no leakage)
- Scaler parameters are computed per fold (no leakage)
- Encoder categories are learned per fold (no leakage)

Without pipelines, you would need to manually refit every preprocessor within each fold.

In [ ]:
# Cross-validation with the full pipeline
# Each fold automatically re-fits all preprocessing steps
cv_scores = cross_val_score(
    full_pipeline, 
    df_mixed,  # raw, unprocessed data
    y_mixed, 
    cv=5, 
    scoring='accuracy'
)

print("5-Fold Cross-Validation with Pipeline:")
print(f"  Fold scores: {cv_scores.round(4)}")
print(f"  Mean accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
print()
print("Each fold independently:")
print("  1. Split data into train/validation")
print("  2. fit_transform(train) for all preprocessors")
print("  3. transform(validation) using train-fitted parameters")
print("  4. Fit model on preprocessed train, evaluate on preprocessed validation")

In [ ]:
# Visualize cross-validation scores
fig, ax = plt.subplots(figsize=(8, 4))
folds = range(1, len(cv_scores) + 1)
ax.bar(folds, cv_scores, color='steelblue', edgecolor='black', alpha=0.8)
ax.axhline(cv_scores.mean(), color='red', linestyle='--', 
           label=f'Mean: {cv_scores.mean():.4f}')
ax.set_xlabel('Fold')
ax.set_ylabel('Accuracy')
ax.set_title('5-Fold Cross-Validation Scores (Pipeline)', fontsize=13)
ax.set_xticks(list(folds))
ax.set_ylim(0, 1)
ax.legend()
plt.tight_layout()
plt.show()

---

## 7. Manual Preprocessing vs Pipeline: A Comparison

Let us build the same model both ways and compare the approaches side by side.

In [ ]:
# Create a fresh dataset for comparison
np.random.seed(42)
n = 400

df_compare = pd.DataFrame({
    'age': np.where(np.random.random(n) < 0.1, np.nan, np.random.normal(35, 10, n)),
    'salary': np.where(np.random.random(n) < 0.12, np.nan, np.random.lognormal(10.8, 0.5, n)),
    'tenure_years': np.random.uniform(0, 15, n),
    'department': np.random.choice(['tech', 'finance', 'ops', 'marketing'], n),
    'performance': np.random.choice(['low', 'medium', 'high'], n, p=[0.2, 0.5, 0.3]),
})
y_compare = (np.random.random(n) < 0.3).astype(int)  # 30% attrition rate

X_tr, X_te, y_tr, y_te = train_test_split(
    df_compare, y_compare, test_size=0.2, random_state=42
)

print("Comparison dataset:")
print(df_compare.head())
print(f"\nShape: {df_compare.shape}")
print(f"Missing: {df_compare.isnull().sum().sum()} total")

In [ ]:
# ============================================
# APPROACH 1: Manual Preprocessing (VERBOSE)
# ============================================

# Step 1: Separate columns
num_cols = ['age', 'salary', 'tenure_years']
cat_cols = ['department', 'performance']

X_tr_manual = X_tr.copy()
X_te_manual = X_te.copy()

# Step 2: Impute numeric columns
num_imputer = SimpleImputer(strategy='median')
X_tr_manual[num_cols] = num_imputer.fit_transform(X_tr_manual[num_cols])
X_te_manual[num_cols] = num_imputer.transform(X_te_manual[num_cols])

# Step 3: Scale numeric columns
scaler_manual = StandardScaler()
X_tr_num_scaled = scaler_manual.fit_transform(X_tr_manual[num_cols])
X_te_num_scaled = scaler_manual.transform(X_te_manual[num_cols])

# Step 4: Encode categorical columns
ohe_manual = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_tr_cat_encoded = ohe_manual.fit_transform(X_tr_manual[cat_cols])
X_te_cat_encoded = ohe_manual.transform(X_te_manual[cat_cols])

# Step 5: Combine
X_tr_final = np.hstack([X_tr_num_scaled, X_tr_cat_encoded])
X_te_final = np.hstack([X_te_num_scaled, X_te_cat_encoded])

# Step 6: Fit model
lr_manual = LogisticRegression(random_state=42, max_iter=1000)
lr_manual.fit(X_tr_final, y_tr)
acc_manual = accuracy_score(y_te, lr_manual.predict(X_te_final))

print(f"Manual approach accuracy: {acc_manual:.4f}")
print(f"Lines of preprocessing code: ~15")
print(f"Objects to serialize for production: 4 (imputer, scaler, encoder, model)")

In [ ]:
# ============================================
# APPROACH 2: Pipeline (CLEAN)
# ============================================

num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

ct = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

pipeline = Pipeline([
    ('preprocessor', ct),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

# Two lines: fit and evaluate
pipeline.fit(X_tr, y_tr)
acc_pipeline = accuracy_score(y_te, pipeline.predict(X_te))

print(f"Pipeline approach accuracy: {acc_pipeline:.4f}")
print(f"Lines of preprocessing code: ~2 (fit + predict)")
print(f"Objects to serialize for production: 1 (the pipeline)")

In [ ]:
# Side-by-side comparison
print("=" * 55)
print(f"{'Metric':<30} {'Manual':>10} {'Pipeline':>10}")
print("=" * 55)
print(f"{'Accuracy':<30} {acc_manual:>10.4f} {acc_pipeline:>10.4f}")
print(f"{'Code lines to preprocess':<30} {'~15':>10} {'~2':>10}")
print(f"{'Objects to serialize':<30} {'4':>10} {'1':>10}")
print(f"{'Leakage risk':<30} {'High':>10} {'None':>10}")
print(f"{'Works with cross_val_score':<30} {'No':>10} {'Yes':>10}")
print("=" * 55)
print("\nSame accuracy, but pipelines are safer and cleaner.")

---

## 8. Common Mistakes

### Mistake 1: Not Using Pipelines in Production

In production, you need to apply the **exact same preprocessing** to new data. Without a pipeline, you must manage multiple objects and risk inconsistencies.

In [ ]:
# Production scenario: new data arrives
new_employee = pd.DataFrame({
    'age': [28],
    'salary': [65000],
    'tenure_years': [3.5],
    'department': ['tech'],
    'performance': ['high']
})

# With pipeline: one call
prediction = pipeline.predict(new_employee)
probability = pipeline.predict_proba(new_employee)

print("Production prediction with pipeline:")
print(f"  Prediction: {'Will leave' if prediction[0] == 1 else 'Will stay'}")
print(f"  Probability: {probability[0][1]:.4f}")
print()
print("No need to separately call imputer, scaler, encoder - the pipeline handles it all.")

### Mistake 2: Fitting a Transformer Outside the Pipeline

If you fit a scaler on the full dataset and then put it in a pipeline, the pipeline does not know about the data leakage.

In [ ]:
# WRONG: Pre-fitting a scaler and putting it in a pipeline
print("WRONG approach:")
print("  scaler = StandardScaler()")
print("  scaler.fit(ALL_DATA)          # leakage!")
print("  pipe = Pipeline([")
print("      ('scaler', scaler),       # already fitted on all data")
print("      ('model', LogisticRegression())")
print("  ])")
print("  pipe.fit(X_train, y_train)    # scaler.fit() called again, but damage done")
print()
print("CORRECT approach:")
print("  pipe = Pipeline([")
print("      ('scaler', StandardScaler()),  # unfitted!")
print("      ('model', LogisticRegression())")
print("  ])")
print("  pipe.fit(X_train, y_train)    # scaler learns from train only")

### Mistake 3: Using cross_val_score Without a Pipeline

If you preprocess first and then use `cross_val_score` on the already-preprocessed data, the scaler/imputer has already seen all folds. This leaks information.

In [ ]:
# WRONG: Preprocess, then cross-validate
X_all_numeric = df_compare[num_cols].copy()

# Impute + scale on ALL data first (leakage)
imp = SimpleImputer(strategy='median')
scl = StandardScaler()
X_preprocessed = scl.fit_transform(imp.fit_transform(X_all_numeric))

# cross_val_score on already-preprocessed data
scores_leaked = cross_val_score(
    LogisticRegression(random_state=42), 
    X_preprocessed, y_compare, cv=5
)

# CORRECT: Use pipeline with cross_val_score
pipe_cv = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(random_state=42))
])
scores_correct = cross_val_score(pipe_cv, X_all_numeric, y_compare, cv=5)

print(f"With leakage (preprocess then CV):   {scores_leaked.mean():.4f} (+/- {scores_leaked.std():.4f})")
print(f"Without leakage (pipeline + CV):     {scores_correct.mean():.4f} (+/- {scores_correct.std():.4f})")
print()
print("The leaked scores may be optimistic.")
print("Always use a Pipeline inside cross_val_score.")

### Summary of Common Mistakes

| Mistake | Consequence | Fix |
|---------|-------------|-----|
| No pipeline in production | Multiple objects, inconsistent preprocessing | Use a single Pipeline |
| Pre-fitted transformer in pipeline | Data leakage | Always put unfitted transformers in the pipeline |
| Preprocessing before cross_val_score | Leakage across folds | Put all preprocessing inside the Pipeline |
| Using `remainder='drop'` without realizing | Columns silently dropped | Set `remainder='passthrough'` if you want to keep extra columns |

---

## 9. Exercise

**Task:** Build a complete pipeline for the dataset below. The dataset simulates customer churn prediction with numeric features (some missing), categorical features, and a binary target.

Requirements:
1. Define numeric and categorical column lists
2. Create sub-pipelines: numeric (median imputer + StandardScaler), categorical (mode imputer + OneHotEncoder)
3. Combine into a ColumnTransformer
4. Chain with LogisticRegression into a full Pipeline
5. Evaluate with 5-fold cross-validation using `cross_val_score`
6. Report the mean accuracy and standard deviation

In [ ]:
# Exercise starter code
np.random.seed(42)
n = 500

exercise_df = pd.DataFrame({
    'monthly_charges': np.where(
        np.random.random(n) < 0.08, np.nan, np.random.normal(65, 30, n)
    ),
    'tenure_months': np.where(
        np.random.random(n) < 0.05, np.nan, np.random.uniform(1, 72, n)
    ),
    'total_charges': np.where(
        np.random.random(n) < 0.1, np.nan, np.random.lognormal(7, 1, n)
    ),
    'contract': np.random.choice(
        ['month-to-month', 'one_year', 'two_year'], n, p=[0.5, 0.3, 0.2]
    ),
    'internet_service': np.random.choice(
        ['dsl', 'fiber_optic', 'none'], n, p=[0.4, 0.4, 0.2]
    ),
    'payment_method': np.random.choice(
        ['electronic_check', 'mailed_check', 'bank_transfer', 'credit_card'], n
    ),
})
exercise_y = (np.random.random(n) < 0.26).astype(int)  # ~26% churn rate

print("Exercise dataset:")
print(exercise_df.head())
print(f"\nShape: {exercise_df.shape}")
print(f"Missing values:\n{exercise_df.isnull().sum()}")
print(f"\nChurn rate: {exercise_y.mean():.2%}")

# YOUR CODE HERE
# Step 1: Define column lists
# num_features = [...]
# cat_features = [...]

# Step 2: Create sub-pipelines
# num_pipeline = Pipeline([...])
# cat_pipeline = Pipeline([...])

# Step 3: ColumnTransformer
# preprocessor = ColumnTransformer([...])

# Step 4: Full Pipeline
# full_pipe = Pipeline([...])

# Step 5: Cross-validation
# cv_scores = cross_val_score(full_pipe, exercise_df, exercise_y, cv=5, scoring='accuracy')

# Step 6: Report results
# print(f"Mean accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")